### Download the Pfam seed sequences 

In [199]:
import re
from collections import defaultdict
import requests
from bs4 import BeautifulSoup

class pfam:
    def __init__(self,file):
        url=f'http://pfam.xfam.org/clan/{file}#tabview=tab2'
        response=requests.get(url)
        soup=BeautifulSoup(response.text,'html.parser')
        self.x=soup.findAll('table',{'summary':'Details of alignments'})
    def table(self):
        self.data=[]
        for h in self.x:
            for i in h.findAll('tr')[1:-1]:
                daa=[]
                for j in i.findAll('td'):
                    daa.append(j.get_text().strip())
                self.data.append(re.findall('(PF\d+)',daa[0]))
        return self.data

class family:
    def __init__(self,idy,chain):
        self.idy=idy
        self.chain=chain
#         print(idy)
    def uniprot(self):
        url1=f'https://www.uniprot.org/uniprot/{self.idy}.fasta'
        response1=requests.get(url1)
        soup1=BeautifulSoup(response1.text,'html.parser')
        xn=soup1.get_text().replace('\n','@',1).replace('\n','').split('@')
        if len(xn)==2:
            return xn[0],xn[1]
        elif len(xn)>2:
            print('Multiple sequences')
            return 0,0
        else:
            print('No sequence found')
            return 0,0
    def pdb(self):
        url2=f'https://www.rcsb.org/fasta/entry/{self.idy}/display'
        response2=requests.get(url2)
        soup2=BeautifulSoup(response2.text,'html.parser')
        x=soup2.get_text()
        if x=='No fasta files were found.':
            print('No fasta files were found.')
        acc,seq=[],[]
        for i in x.split('\n'):
            if i.startswith('>'):
                acc.append(i)
            else:
                seq.append(i)
        try:
            for j in range(len(acc)):
                if re.search(f'Chains .*{self.chain}',acc[j].split('|')[1]):
                    aa,s=acc[j],seq[j]
            return aa,s
        except UnboundLocalError:
            print(f'{self.chain} Chain not found in {self.idy}')
            return 0,0

class collect:
    def __init__(self,file):
        self.file=file
        url=f'http://pfam.xfam.org/family/{file}/alignment/seed'
        response=requests.get(url)
        soup=BeautifulSoup(response.text,'html.parser')
        self.x=[re.split('\s+',z)[1:] for z in list(filter(lambda y:y.startswith('#=GS'),soup.get_text().split('\n')))]
        self.temp=defaultdict(list)
        for each in self.x:
            self.temp[each[0]].append('$'.join(each[1:]))
            
    def output(self):
        out=open(f'{self.file}_sequences_{len(self.temp)}.txt','w')
        for m in self.temp.values():
            for each in m:
                if each.startswith('DR'):
                    jk=each.split('$')
                    a,b=family(jk[2],jk[3][:-1]).pdb()
                else:
                    a,b=family(each.split('$')[1],0).uniprot()

                if a!=0 and b!=0:
                        out.write(a+'\n')
                        out.write(b+'\n')
                        break
        out.close()
        print(f'{len(self.temp)} sequences collected for {self.file}')
        return None

In [200]:
all_fam=pfam('CL0037').table()

In [205]:
for ee in all_fam:
    if len(ee)>1:
        print('Check for the multiple pfam families')
    else:
        collect(ee[0]).output()
print('Mission completed......')

26 sequences collected for PF01464
108 sequences collected for PF00912
163 sequences collected for PF13406
539 sequences collected for PF01832
6 sequences collected for PF00182
15 sequences collected for PF06737
14 sequences collected for PF00959
12 sequences collected for PF00062
161 sequences collected for PF03998
98 sequences collected for PF05838
56 sequences collected for PF19489
32 sequences collected for PF01374
56 sequences collected for PF18013
18 sequences collected for PF13702
46 sequences collected for PF05497
16 sequences collected for PF18896
5 sequences collected for PF16754
2 sequences collected for PF06871
1 sequences collected for PF10464
1 sequences collected for PF11508
3 sequences collected for PF13647
Mission completed......
